# Homework 3 - MongoDB

In [1]:
import json
import os
import pymongo
from pymongo import ASCENDING, DESCENDING
import pandas as pd
from datetime import date, datetime, timedelta

# Lettura dei dati

In [2]:
dataset_dir = './dataset/'
dati_covid_path = os.path.join(dataset_dir, 'dpc-covid19-ita-regioni.json')
dati_vaccini_path = os.path.join(dataset_dir, 'somministrazioni-vaccini-latest.json')

dati_covid = []
with open(dati_covid_path, 'r') as j:
    dati_covid = json.load(j)

dati_vaccini = []
with open(dati_vaccini_path, 'r') as j:
    dati_vaccini = json.load(j)
dati_vaccini = dati_vaccini['data']

# Creazione del database
Si crea il database e due collection contenenti rispettivamente dati relativi al Covid-19 e alle vaccinazioni.

In [3]:
client = pymongo.MongoClient('localhost', 27017)

db = client['homework3']

collection_covid = db.dati_covid
collection_vaccini = db.dati_vaccini

print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'local']
Available collections:  []


Si inseriscono i dati nelle rispettive collection (solo se il database è stato appena creato).

In [4]:
# collection_covid.insert_many(dati_covid)
# collection_vaccini.insert_many(dati_vaccini)

In [5]:
print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'homework3', 'local']
Available collections:  ['dati_covid', 'dati_vaccini']


In [6]:
collection_covid.find_one()

{'_id': ObjectId('60aa2da2256e0fad9c05adef'),
 'data': '2020-02-24T18:00:00',
 'stato': 'ITA',
 'codice_regione': 13,
 'denominazione_regione': 'Abruzzo',
 'lat': 42.35122196,
 'long': 13.39843823,
 'ricoverati_con_sintomi': 0,
 'terapia_intensiva': 0,
 'totale_ospedalizzati': 0,
 'isolamento_domiciliare': 0,
 'totale_positivi': 0,
 'variazione_totale_positivi': 0,
 'nuovi_positivi': 0,
 'dimessi_guariti': 0,
 'deceduti': 0,
 'casi_da_sospetto_diagnostico': None,
 'casi_da_screening': None,
 'totale_casi': 0,
 'tamponi': 5,
 'casi_testati': None,
 'note': None,
 'ingressi_terapia_intensiva': None,
 'note_test': None,
 'note_casi': None,
 'totale_positivi_test_molecolare': None,
 'totale_positivi_test_antigenico_rapido': None,
 'tamponi_test_molecolare': None,
 'tamponi_test_antigenico_rapido': None,
 'codice_nuts_1': None,
 'codice_nuts_2': None}

In [7]:
collection_vaccini.find_one()

{'_id': ObjectId('60aa2da4256e0fad9c05d32d'),
 'index': 0,
 'data_somministrazione': '2020-12-27T00:00:00.000Z',
 'fornitore': 'Pfizer/BioNTech',
 'area': 'ABR',
 'fascia_anagrafica': '20-29',
 'sesso_maschile': 1,
 'sesso_femminile': 0,
 'categoria_operatori_sanitari_sociosanitari': 1,
 'categoria_personale_non_sanitario': 0,
 'categoria_ospiti_rsa': 0,
 'categoria_60_69': 0,
 'categoria_70_79': 0,
 'categoria_over80': 0,
 'categoria_forze_armate': 0,
 'categoria_personale_scolastico': 0,
 'categoria_soggetti_fragili': 0,
 'categoria_altro': 0,
 'prima_dose': 1,
 'seconda_dose': 0,
 'codice_NUTS1': 'ITF',
 'codice_NUTS2': 'ITF1',
 'codice_regione_ISTAT': 13,
 'nome_area': 'Abruzzo'}

# Preprocessing
Si convertono i campi relativi alle date dal formato `string` al formato `date`.

In [8]:
filter = {}

update = [{"$set": {
            'data_somministrazione': {
                "$toDate": "$data_somministrazione"
                }
             }
         }]

collection_vaccini.update_many(filter, update)

In [9]:
filter = {}

update = [{"$set": {
            'data': {
                "$toDate": "$data"
                }
             }
         }]

collection_covid.update_many(filter, update)

# Views and queries

## Dati Covid-19

### V1: Totale casi e deceduti ad ogni mese

In [10]:
pipeline = [
    { "$group" : {
        "_id": {
            "anno" : {"$year": "$data"},
            "mese" : {"$month": "$data"},
            "regione" : "$denominazione_regione"
        },
        "deceduti": { "$max": "$deceduti" },
        "casi": { "$max": "$totale_casi" }
    }},
    
    { "$group" : {
        "_id": {
            "anno" : "$_id.anno",
            "mese" : "$_id.mese",
        },
        "deceduti": { "$sum": "$deceduti" },
        "casi": { "$sum": "$casi" }
    }},
        
    { "$project": {
        "anno": "$_id.anno",
        "mese": "$_id.mese",
        "deceduti": 1,
        "casi": 1,
        "_id": 0
    }},
    
    { "$sort" : {"anno": -1, "mese": -1} }
]

db.create_collection(
    'dati_covid_mensili',
    viewOn='dati_covid',
    pipeline=pipeline
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'homework3'), 'dati_covid_mensili')

In [11]:
result = db.dati_covid_mensili.find()
result_list = list(result)

df = pd.DataFrame(result_list)
df

,deceduti,casi,anno,mese
0,125153,4188190,2021,5
1,120807,4022653,2021,4
2,109346,3583444,2021,3
3,97699,2925265,2021,2
4,88516,2553032,2021,1
5,74159,2107166,2020,12
6,55576,1601554,2020,11
7,38618,679430,2020,10
8,35894,314861,2020,9
9,35483,269214,2020,8


### Q3: Totale casi e deceduti ad ogni mese nel 2021

In [12]:
result = db.dati_covid_mensili.find(filter={'anno': 2021})
result_list = list(result)

df = pd.DataFrame(result_list)
df

,deceduti,casi,anno,mese
0,125153,4188190,2021,5
1,120807,4022653,2021,4
2,109346,3583444,2021,3
3,97699,2925265,2021,2
4,88516,2553032,2021,1


## Dati vaccini

### V2: Prima e seconda dose per regione e fascia anagrafica

In [13]:
pipeline = [
    { "$group": {
        "_id": {
            "regione": "$nome_area",
            "fascia_anagrafica": "$fascia_anagrafica"
        },
        "prima_dose": { "$sum": "$prima_dose" },
        "seconda_dose" : { "$sum": "$seconda_dose"}
    }},
    
    { "$project": {
        "regione": "$_id.regione",
        "fascia_anagrafica": "$_id.fascia_anagrafica",
        "prima_dose": 1,
        "seconda_dose": 1,
        "_id": 0
    }},
    
    { "$sort" : { "regione": 1,  "fascia_anagrafica": 1} }
]

db.create_collection(
    'dati_vaccini_regioni_fasce',
    viewOn='dati_vaccini',
    pipeline=pipeline
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'homework3'), 'dati_vaccini_regioni_fasce')

In [14]:
result = db.dati_vaccini_regioni_fasce.find()
result_list = list(result)

df = pd.DataFrame(result_list)
df.head(20)

,prima_dose,seconda_dose,regione,fascia_anagrafica
0,1108,521,Abruzzo,16-19
1,15949,7344,Abruzzo,20-29
2,24562,12656,Abruzzo,30-39
3,44627,18868,Abruzzo,40-49
4,70655,28371,Abruzzo,50-59
5,104834,35544,Abruzzo,60-69
6,108091,42611,Abruzzo,70-79
7,74466,65802,Abruzzo,80-89
8,17799,14516,Abruzzo,90+
9,1456,414,Basilicata,16-19


### Q4: Prima e seconda dose per regione

In [15]:
result = db.dati_vaccini_regioni_fasce.aggregate([
    { "$group": {
        "_id": "$regione",
        "prima_dose": { "$sum": "$prima_dose" },
        "seconda_dose" : { "$sum": "$seconda_dose"}
    }},
    
    { "$sort" : { "prima_dose": -1 } }
])

result_list = list(result)

In [16]:
df = pd.DataFrame(result_list)
df

,_id,prima_dose,seconda_dose
0,Lombardia,3609626,1655555
1,Lazio,2041929,895460
2,Campania,2034829,814263
3,Veneto,1747921,768710
4,Emilia-Romagna,1563154,783585
5,Sicilia,1490197,719623
6,Piemonte,1477529,767950
7,Puglia,1430250,637612
8,Toscana,1230896,630477
9,Calabria,624727,276600


### Q5: Prima e seconda dose per fascia anagrafica

In [17]:
result = db.dati_vaccini_regioni_fasce.aggregate([
    { "$group": {
        "_id": "$fascia_anagrafica",
        "prima_dose": { "$sum": "$prima_dose" },
        "seconda_dose" : { "$sum": "$seconda_dose"}
    }},
    
    { "$sort" : { "prima_dose": -1 } }
])

result_list = list(result)

In [18]:
df = pd.DataFrame(result_list)
df

,_id,prima_dose,seconda_dose
0,60-69,4863518,1323231
1,70-79,4809466,1612266
2,50-59,3402991,1206718
3,80-89,3311967,2974452
4,40-49,1767644,842068
5,30-39,1083518,579357
6,20-29,799809,412516
7,90+,757214,651397
8,16-19,67913,24579


### V3: Dosi somministrate per regione e categoria

In [19]:
pipeline = [
    { "$group": {
        "_id": "$nome_area",
        "personale_scolastico": { "$sum": "$categoria_personale_scolastico" },
        "soggetti_fragili": { "$sum": "$categoria_soggetti_fragili" },
        "operatori_sanitari_sociosanitari": { "$sum": "$categoria_operatori_sanitari_sociosanitari" },
        "personale_non_sanitario": { "$sum": "$categoria_personale_non_sanitario" },
        "60_69": { "$sum": "$categoria_60_69" },
        "over80": { "$sum": "$categoria_over80" },
        "altro": { "$sum": "$categoria_altro" },
        "ospiti_rsa": { "$sum": "$categoria_ospiti_rsa" },
        "70_79": { "$sum": "$categoria_70_79" },
        "forze_armate": { "$sum": "$categoria_forze_armate" }
    }},   
    
    { "$project": {
        "_id": 0,
        "regione": "$_id",
        "personale_scolastico": 1,
        "soggetti_fragili": 1,
        "operatori_sanitari_sociosanitari": 1,
        "personale_non_sanitario": 1,
        "60_69": 1,
        "over80": 1,
        "altro": 1,
        "ospiti_rsa": 1,
        "70_79": 1,
        "forze_armate": 1
    }}
]

db.create_collection(
    'dati_vaccini_regioni_categorie',
    viewOn='dati_vaccini',
    pipeline=pipeline
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'homework3'), 'dati_vaccini_regioni_categorie')

In [20]:
result = db.dati_vaccini_regioni_categorie.find()
result_list = list(result)

df = pd.DataFrame(result_list)
df.head(20)

,personale_scolastico,soggetti_fragili,operatori_sanitari_sociosanitari,personale_non_sanitario,60_69,over80,altro,ospiti_rsa,70_79,forze_armate,regione
0,16699,104268,26297,7080,16111,71999,14553,4898,26901,5653,Basilicata
1,147138,252564,336156,24363,451261,691227,335640,44606,584940,69494,Lazio
2,32012,304997,90462,31148,75889,148624,80595,29332,85576,22692,Calabria
3,103591,335688,254730,74206,112965,568675,68183,39661,269232,34442,Toscana
4,2351,8242,7186,2186,10538,14423,3933,1556,12742,1764,Valle d'Aosta / Vallée d'Aoste
5,217090,815245,197164,43270,281009,446186,425920,16817,366258,40133,Campania
6,112288,315453,280054,75267,391849,556582,225881,84838,451443,22976,Veneto
7,215511,828058,598833,233731,825154,1231473,164165,143819,962883,61554,Lombardia
8,173781,465404,242100,53323,256166,413695,83553,30903,316345,32592,Puglia
9,12530,19775,30894,7837,41416,58301,37169,8549,44690,4650,Provincia Autonoma Trento


### Q6: Dosi somministrate per categoria

In [21]:
result = db.dati_vaccini_regioni_categorie.aggregate([
    { "$group": {
        "_id": "null",
        "personale_scolastico": { "$sum": "$personale_scolastico" },
        "soggetti_fragili": { "$sum": "$soggetti_fragili" },
        "operatori_sanitari_sociosanitari": { "$sum": "$operatori_sanitari_sociosanitari" },
        "personale_non_sanitario": { "$sum": "$personale_non_sanitario" },
        "60_69": { "$sum": "$60_69" },
        "over80": { "$sum": "$80" },
        "altro": { "$sum": "$altro" },
        "ospiti_rsa": { "$sum": "$ospiti_rsa" },
        "70_79": { "$sum": "$70_79" },
        "forze_armate": { "$sum": "$forze_armate" }
    }},
    
    { "$project": {
        "_id": 0
    }}
])

result_list = list(result)

In [22]:
df = pd.DataFrame(result_list)
df

,personale_scolastico,soggetti_fragili,operatori_sanitari_sociosanitari,personale_non_sanitario,60_69,over80,altro,ospiti_rsa,70_79,forze_armate
0,1557866,6422747,3427335,988025,3483209,0,1970840,693699,4551183,480331


### V4: Dosi somministrate per fascia anagrafica e fornitore

In [23]:
pipeline = [
    { "$group" : {
        "_id": {
            "fascia_anagrafica": "$fascia_anagrafica",
            "fornitore": "$fornitore"
        },
        "prima_dose": { "$sum": "$prima_dose" },
        "seconda_dose" : { "$sum": "$seconda_dose"}
    }},
    
    { "$addFields": {
        "totale_dosi": { "$sum": ["$prima_dose", "$seconda_dose"]}
    }},
    
    { "$project": {
        "fascia_anagrafica": "$_id.fascia_anagrafica",
        "fornitore": "$_id.fornitore",
        "totale_dosi": 1,
        "_id": 0
    }},
    
    { "$sort" : {
        "fascia_anagrafica": -1, 
        "totale_dosi": -1
    }}
]

db.create_collection(
    'dati_vaccini_fasce_fornitori',
    viewOn='dati_vaccini',
    pipeline=pipeline
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'homework3'), 'dati_vaccini_fasce_fornitori')

In [24]:
result = db.dati_vaccini_fasce_fornitori.find()
result_list = list(result)

df = pd.DataFrame(result_list)
df

,totale_dosi,fascia_anagrafica,fornitore
0,1166414,90+,Pfizer/BioNTech
1,232017,90+,Moderna
2,8622,90+,Vaxzevria (AstraZeneca)
3,1558,90+,Janssen
4,5548385,80-89,Pfizer/BioNTech
5,667780,80-89,Moderna
6,67273,80-89,Vaxzevria (AstraZeneca)
7,2981,80-89,Janssen
8,3655678,70-79,Pfizer/BioNTech
9,2162794,70-79,Vaxzevria (AstraZeneca)


### Q7: Principale fornitore per fascia anagrafica

In [25]:
result = db.dati_vaccini_fasce_fornitori.aggregate([
    { "$group" : {
        "_id": "$fascia_anagrafica",
        "totale_dosi": {"$first": "$totale_dosi"},
        "fornitore": {"$first": "$fornitore" }
    }}, 
    
    { "$sort" : {
        "_id": -1, 
    }},
])

result_list = list(result)

In [26]:
df = pd.DataFrame(result_list)
df

,_id,totale_dosi,fornitore
0,90+,1166414,Pfizer/BioNTech
1,80-89,5548385,Pfizer/BioNTech
2,70-79,3655678,Pfizer/BioNTech
3,60-69,3674368,Pfizer/BioNTech
4,50-59,3284392,Pfizer/BioNTech
5,40-49,1734724,Pfizer/BioNTech
6,30-39,1144658,Pfizer/BioNTech
7,20-29,888774,Pfizer/BioNTech
8,16-19,80181,Pfizer/BioNTech


### V5: Dosi somministrate per fascia anagrafica ogni mese

In [27]:
pipeline = [
    { "$group" : {
        "_id": {
            "anno" : {"$year": "$data_somministrazione"},
            "mese" : {"$month": "$data_somministrazione"},
            "fascia_anagrafica" : "$fascia_anagrafica"
        },
        "prima_dose": { "$sum": "$prima_dose" },
        "seconda_dose" : { "$sum": "$seconda_dose"}
    }},
    
    { "$addFields": {
        "totale_dosi": { "$sum": ["$prima_dose", "$seconda_dose"]}
    }},    
        
    { "$project": {
        "anno": "$_id.anno",
        "mese": "$_id.mese",
        "fascia_anagrafica": "$_id.fascia_anagrafica",
        "totale_dosi": 1,
        "_id": 0
    }},
    
    { "$sort" : {
        "anno": -1,
        "mese": -1,        
        "totale_dosi": -1
    }}
]

db.create_collection(
    'dati_vaccini_fasce_mensili',
    viewOn='dati_vaccini',
    pipeline=pipeline
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'homework3'), 'dati_vaccini_fasce_mensili')

In [28]:
result = db.dati_vaccini_fasce_mensili.find()
result_list = list(result)

df = pd.DataFrame(result_list)
df

,totale_dosi,anno,mese,fascia_anagrafica
0,3309114,2021,5,60-69
1,2253686,2021,5,70-79
2,2241462,2021,5,50-59
3,904944,2021,5,40-49
4,685521,2021,5,80-89
5,455194,2021,5,30-39
6,354197,2021,5,20-29
7,168715,2021,5,90+
8,54182,2021,5,16-19
9,3411580,2021,4,70-79


### Q8: Fascia anagrafica più vaccinata ogni mese

In [29]:
result = db.dati_vaccini_fasce_mensili.aggregate([    
    { "$group" : {
        "_id": {
            "anno": "$anno",
            "mese": "$mese"
        },
        "totale_dosi": {"$first": "$totale_dosi"},
        "fascia_anagrafica": {"$first": "$fascia_anagrafica" }
    }}, 
    
    { "$sort" : {
        "_id": -1, 
    }},
])

result_list = list(result)

In [30]:
df = pd.DataFrame(result_list)
df

,_id,totale_dosi,fascia_anagrafica
0,"{'anno': 2021, 'mese': 5}",3309114,60-69
1,"{'anno': 2021, 'mese': 4}",3411580,70-79
2,"{'anno': 2021, 'mese': 3}",2425520,80-89
3,"{'anno': 2021, 'mese': 2}",613806,80-89
4,"{'anno': 2021, 'mese': 1}",523279,50-59
5,"{'anno': 2020, 'mese': 12}",11268,50-59


# Indexes

In [31]:
print(list(collection_covid.list_indexes()))

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])]


## Query without index

### Q1: Attuali positivi e terapie intensive per regione

In [58]:
days_ago = 1
date = datetime.today() - timedelta(days=days_ago)

start = date.replace(hour=0, minute=0, second=0)
end = date.replace(hour=23, minute=59, second=59)

filter = {
    'data': {
        '$gt': start,
        '$lt': end
    }
}

project = {
    'data': 1, 
    'denominazione_regione': 1, 
    'totale_positivi': 1,
    'terapia_intensiva': 1,
    '_id': 0
}

sort = list({
    'totale_positivi': -1
}.items())

result = collection_covid.find(
    filter=filter,
    projection=project,
    sort=sort
)

result_list = list(result)

explain_no_ind = result.explain()
explain_no_ind = explain_no_ind['executionStats']

In [59]:
df = pd.DataFrame(result_list)
df

,data,denominazione_regione,terapia_intensiva,totale_positivi
0,2021-05-22 17:00:00,Campania,82,72531
1,2021-05-22 17:00:00,Lombardia,298,35416
2,2021-05-22 17:00:00,Puglia,100,32794
3,2021-05-22 17:00:00,Lazio,195,28334
4,2021-05-22 17:00:00,Emilia-Romagna,126,19338
5,2021-05-22 17:00:00,Sardegna,32,13484
6,2021-05-22 17:00:00,Sicilia,104,13267
7,2021-05-22 17:00:00,Toscana,144,11552
8,2021-05-22 17:00:00,Veneto,72,11296
9,2021-05-22 17:00:00,Calabria,24,11173


In [60]:
print('executionTimeMillis:', explain_no_ind['executionTimeMillis'])
print('nReturned:', explain_no_ind['nReturned'])
print('totalDocsExamined:', explain_no_ind['totalDocsExamined'])

executionTimeMillis: 16
nReturned: 21
totalDocsExamined: 9534


## Query with index

In [68]:
collection_covid.create_index([("data", ASCENDING)], 
                              name="date_index",
                              background=True,
                              unique=False,
                              sparse=False)

'date_index'

In [38]:
print(list(collection_covid.list_indexes()))

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]), SON([('v', 2), ('key', SON([('data', 1)])), ('name', 'date_index'), ('background', True), ('sparse', False)])]


### Q1: Attuali positivi e terapie intensive per regione

In [73]:
days_ago = 1
date = datetime.today() - timedelta(days=days_ago)

start = date.replace(hour=0, minute=0, second=0)
end = date.replace(hour=23, minute=59, second=59)

filter = {
    'data': {
        '$gt': start,
        '$lt': end
    }
}

project = {
    'data': 1, 
    'denominazione_regione': 1, 
    'totale_positivi': 1,
    'terapia_intensiva': 1,
    '_id': 0
}

sort = list({
    'totale_positivi': -1
}.items())

result = collection_covid.find(
    filter=filter,
    projection=project,
    sort=sort
)

result_list = list(result)

explain_ind = result.explain()
explain_ind = explain_ind['executionStats']

In [75]:
print('executionTimeMillis:', explain_ind['executionTimeMillis'])
print('nReturned:', explain_ind['nReturned'])
print('totalDocsExamined:', explain_ind['totalDocsExamined'])
print('indexName:', explain_ind['executionStages']['inputStage']['inputStage']['inputStage']['indexName'])

executionTimeMillis: 0
nReturned: 21
totalDocsExamined: 21
indexName: date_index


### Q2: Nuovi casi e tamponi nell'ultima settimana per regione

In [79]:
weeks_ago = 1
date = datetime.today() - timedelta(weeks=weeks_ago)

result = collection_covid.aggregate([
    { "$match": { "data": { "$gte": date } } },

    { "$group": {
        "_id": "$denominazione_regione",
        "nuovi_casi": { "$sum": "$nuovi_positivi" },
        "tamponi" : { "$sum": "$tamponi"}
    }},
    
    { "$sort" : { "nuovi_casi": -1 } }
])

result_list = list(result)

In [80]:
df = pd.DataFrame(result_list)
df

,_id,nuovi_casi,tamponi
0,Lombardia,5683,71756835
1,Campania,4620,33139697
2,Lazio,3302,46866103
3,Sicilia,3004,30165040
4,Toscana,2829,31683802
5,Piemonte,2781,30236080
6,Emilia-Romagna,2669,42205861
7,Puglia,2539,16840095
8,Veneto,1890,53830701
9,Calabria,1308,5847161


In [76]:
weeks_ago = 1
date = datetime.today() - timedelta(weeks=weeks_ago)

result = db.command('aggregate', 'dati_covid', pipeline=[
    { "$match": { "data": { "$gte": date } } },

    { "$group": {
        "_id": "$denominazione_regione",
        "nuovi_casi": { "$sum": "$nuovi_positivi" },
        "tamponi" : { "$sum": "$tamponi"}
    }},
    
    { "$sort" : { "nuovi_casi": -1 } }
], explain=True)

result_list = list(result)

In [77]:
result

{'stages': [{'$cursor': {'queryPlanner': {'plannerVersion': 1,
     'namespace': 'homework3.dati_covid',
     'indexFilterSet': False,
     'parsedQuery': {'data': {'$gte': datetime.datetime(2021, 5, 16, 13, 14, 44, 735000)}},
     'queryHash': '5EBD9AEC',
     'planCacheKey': '0D44E561',
     'winningPlan': {'stage': 'PROJECTION_SIMPLE',
      'transformBy': {'denominazione_regione': 1,
       'nuovi_positivi': 1,
       'tamponi': 1,
       '_id': 0},
      'inputStage': {'stage': 'FETCH',
       'inputStage': {'stage': 'IXSCAN',
        'keyPattern': {'data': 1},
        'indexName': 'date_index',
        'isMultiKey': False,
        'multiKeyPaths': {'data': []},
        'isUnique': False,
        'isSparse': False,
        'isPartial': False,
        'indexVersion': 2,
        'direction': 'forward',
        'indexBounds': {'data': ['[new Date(1621170884735), new Date(9223372036854775807)]']}}}},
     'rejectedPlans': []}}},
  {'$group': {'_id': '$denominazione_regione',
    'nuov